In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import random
import os
import datetime


In [2]:
stock ={"陽明":"2609","長榮":"2603","萬海":"2615","長榮航":"2618","遠雄港":"5607","華航":"2610","台驊投控":"2636","裕民":"2606",
        "益航":"2601" ,"東森":"2614","新興":"2605" , "四維航":"5608" , "台航":"2617" }
month_now =datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m')+"01" #現在的月份，url所使用
month_index = datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m') #index使用
url_PBR = [f"https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date={month_now}&stockNo={code}" for code in stock.values() ]


In [3]:
#header
headers ={"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

In [4]:
def get_text(url   ,headers ):
    """負責request的請求
    傳入網址、header(避免反爬蟲機制)"""
    r = requests.get(url, headers=headers )
    res = r.text
    soup=BeautifulSoup( res ,"lxml")
    
    return soup.table

In [5]:
def get_value_PBR(data):
    """請傳入 rqquest.get(url).text"""
    
    """return 資料為一個月的數值的 PBR 股價淨值比 """
    #使用BeautifulSoup 做解析
    
    df=pd.read_html(data.prettify())[0]
    df.columns = df.columns.get_level_values(1)
    PBR_end=np.median(df["股價淨值比"].map(lambda x : float(x))) #找出當月股價淨值比的中位數

    return   PBR_end

In [6]:
df_plus=[]
for url_pbr  in url_PBR :
    print(url_pbr)
    data= get_text(url_pbr ,headers )
    time.sleep(3)
    while data is None:
        data= get_text(url_pbr ,headers )
        
    PBR_end =get_value_PBR(data)

    df=pd.DataFrame({"股價淨值比":PBR_end },index= [month_index]).round(2)
    df_plus.append(df)

https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2603
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2615
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2618
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=5607
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2610
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2636
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2606
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2601
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2614
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20210601&stockNo=2605
https://www.twse.com.tw/exchangeReport/BWIB

In [7]:
#將新的資訊加入原有的資料中
for new ,  key ,name in zip(df_plus  , stock.keys() ,stock.values()) :
    #1.新比對月份是否有重複的
    old=pd.read_csv(f"./Data_股價淨值比/{key}_{name}.csv",index_col=["Unnamed: 0"])#讀入舊的dataFRAME
    new_index=int(new.index.values)
    old_index=old.tail(1).index.values
    if new_index == old_index: 
        """檢查最後ㄧ列的月份是否相同，如相同drop掉，用新的資料"""
        old.drop(index =old_index ,inplace=True )
        #2.將新的月份資料結合
        new_= pd.concat([old ,new])
        
    else:
         #2.將新的月份資料結合
        new_= pd.concat([old ,new])
    
    new_.to_csv(f"./Data_股價淨值比/{key}_{name}.csv")
        